## Instructions for reproduction -

I expect you'd want to run this on a cluster like MAMBA. You'll need two scripts to submit your job -

`do.sh` - https://paste.rs/
1a4

```
#!/bin/bash

#rm -rf semisupervised_nlu

git clone --recurse-submodules https://github.com/DhruvDh/semisupervised_nlu.git

job=$(qsub -d `pwd` -l nodes=1:ppn=16:gpus=1 submit.sh -q mamba -l walltime=11:30:00)
num=${job:0:5}

echo "Job ID: $num"
echo $num > lastjob
```

`submit.sh` - https://paste.rs/xxK

```
#!/bin/bash

export PATH=~/.local/bin:$PATH

#module load cuda/8.0 cudnn/6.0-cuda8 anaconda3/5.0.1-cuda8

module load pytorch/1.2.0-anaconda3-cuda10.0 
python3 -m pip install --user transformers tensorboardx

source ~/.bashrc
conda init
source ~/.bashrc

cd semisupervised_nlu

python3 ./transformers/examples/run_lm_finetuning.py \
    --output_dir=output \
    --model_type=roberta \
    --model_name_or_path=roberta-base \
    --do_train \
    --train_data_file="./data/roberta/train.txt" \
    --do_eval \
    --eval_data_file="./data/roberta/test.txt" \
    --num_train_epochs=15 \
    --save_steps=659 \
    --save_total_limit=2 \
    --mlm
```

And run `do.sh` to submit the training job. This will produce a model in `semisupervised_nlu/output`, which is where this notebook will try to load the model from later.

In [1]:
from transformers import RobertaForMaskedLM, RobertaTokenizer
import torch
from tqdm import tqdm

## Finetuned performance (only 15 epochs)

In [2]:
import os
import pandas as pd
from random import choice

import nltk

In [3]:
path_to_intents = os.path.join('data', 'raw')
intents = os.listdir(path_to_intents)
get_path = lambda x: os.path.join('data', 'raw', x, x + ".csv")

In [4]:
os.chdir('helpers')
from analyze import questions, entities, get_data
data = get_data()
os.chdir('..')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
model = RobertaForMaskedLM.from_pretrained('./output/')
tokenizer = RobertaTokenizer.from_pretrained('./output/')

In [6]:
from collections import defaultdict
text = defaultdict(list)
encoded_text = defaultdict(list)

ans = defaultdict(list)
response = defaultdict(list)

scores = defaultdict(list)

for (_questions, intent, entity) in zip(questions, intents, entities):
    question = choice(_questions)
    
    for (r, e) in zip(data[intent]['df']['text'], data[intent]['df'][entity]):
        text[intent].append("<s> " + r.strip() + '. ' + question.strip() + " ".join(["<mask>" for x in str(e).split()]) + " </s>")
        ans[intent].append((str(e)).split())

In [7]:
for intent in intents:
    for row in text[intent]:
        encoded_text[intent].append(torch.tensor([tokenizer.encode(row, add_special_tokens=False)]))

In [8]:
with torch.no_grad():
    for intent in intents:
        for i, row in tqdm(enumerate(encoded_text[intent])):
            out = model(row)
            response[intent].append(tokenizer.decode(torch.argmax(out[0][0], dim=1).tolist()).split())
            
            score = nltk.translate.bleu_score.sentence_bleu(
                    [ans[intent][i]],
                    response[intent][i],
                    smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method4,
                    auto_reweigh=True
                )
            scores[intent].append(score)
            

2042it [03:13, 10.56it/s]
2073it [02:58, 11.59it/s]
2100it [03:39,  9.55it/s]
2100it [02:40, 13.08it/s]
2056it [02:41, 12.72it/s]
2054it [02:47, 12.25it/s]
2059it [03:01, 11.33it/s]


In [9]:
for intent in intents:
    print(f"For {intent}, avg. BLEU score is {sum(scores[intent]) / len(scores[intent])}")

For AddToPlaylist, avg. BLEU score is 0.160211052589337
For BookRestaurant, avg. BLEU score is 0.1656835101344695
For GetWeather, avg. BLEU score is 0.0992932125200511
For PlayMusic, avg. BLEU score is 0.12332243402123681
For RateBook, avg. BLEU score is 0.13669591041649146
For SearchCreativeWork, avg. BLEU score is 0.20891380704355164
For SearchScreeningEvent, avg. BLEU score is 0.20672132657334064
